In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import matplotlib.pyplot as plt

In [ ]:
# Load and prepare data
df = pd.read_csv("../data/synthetic_data.csv")
features = ['amount', 'is_international', 'merchant_id']
target = 'is_fraud'

In [ ]:
X = df[features].values.astype(np.float32)
y = df[target].values.astype(np.int32)

In [ ]:
# Shuffle and simulate 3 clients
def simulate_clients(X, y, num_clients=3):
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    X = X[indices]
    y = y[indices]
    X_split = np.array_split(X, num_clients)
    y_split = np.array_split(y, num_clients)
    return [(X_split[i], y_split[i]) for i in range(num_clients)]

In [ ]:
clients_data = simulate_clients(X, y, num_clients=3)

In [ ]:
def create_tf_dataset(X, y):
    return tf.data.Dataset.from_tensor_slices((X, y)).shuffle(buffer_size=10).batch(4)

In [ ]:
federated_train_data = [create_tf_dataset(Xi, yi) for Xi, yi in clients_data]

In [ ]:
# Define the model function
def model_fn():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(len(features),)),
        tf.keras.layers.Dense(8, activation='relu'),
        tf.keras.layers.Dense(4, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return tff.learning.models.from_keras_model(
        keras_model=model,
        input_spec=federated_train_data[0].element_spec,
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

In [ ]:
# Federated training setup
fed_avg = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn=model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.01),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0)
)

In [ ]:
# Initialize state
state = fed_avg.initialize()

In [ ]:
# Run training rounds
rounds = 10
losses, accuracies = [], []

In [ ]:
for round_num in range(1, rounds + 1):
    result = fed_avg.next(state, federated_train_data)
    state = result.state
    metrics = result.metrics['client_work']['train']
    losses.append(metrics['loss'])
    accuracies.append(metrics['binary_accuracy'])
    print(f"Round {round_num}: Loss = {metrics['loss']:.4f}, Accuracy = {metrics['binary_accuracy']:.4f}")

In [ ]:
# Plot metrics
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(range(1, rounds + 1), losses, marker='o')
plt.title("Federated Training Loss")
plt.xlabel("Round")
plt.ylabel("Loss")

In [ ]:
plt.subplot(1, 2, 2)
plt.plot(range(1, rounds + 1), accuracies, marker='o', color='green')
plt.title("Federated Training Accuracy")
plt.xlabel("Round")
plt.ylabel("Accuracy")
plt.tight_layout()
plt.show()

In [ ]:
# Evaluate final model centrally
def evaluate_global_model(state, X, y):
    keras_model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(len(features),)),
        tf.keras.layers.Dense(8, activation='relu'),
        tf.keras.layers.Dense(4, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    tff.learning.models.assign_weights_to_keras_model(keras_model, state.model.trainable)
    keras_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    results = keras_model.evaluate(X, y, verbose=0)
    print(f"Global Model Evaluation: Loss = {results[0]:.4f}, Accuracy = {results[1]:.4f}")

In [ ]:
evaluate_global_model(state, X, y)